<a href="https://colab.research.google.com/github/decoz/mlclass/blob/master/ch4_cnn_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Nerural Network (CNN) 

cnn 은 주로 이미지분야에 사용되는 네트워크로서 사실 딥러닝 혁명을 이끈 주역이라고 할 수 있다. 그전까지는 지나치게 많은 계산량과 복잡한 상태공간으로 인해 난항을 격고 있던 이미지 인식분야에 합성곱 계층과 맥스풀링 계층을 도입하여 이 문제를 해결하는 기초를 제공하였다. 

<hr style="height:2px" > 


## 간단한 cnn 모델 만들어보기 

### 합성곱 계층

합성곱 신경망은 특정 블럭형태의 필터를 이미지에 슬라이딩하면서 적용하여 그 값을 매트릭스형태로 출력한다. 이전 수업에서 배운 가우시안 필터를 기억하는 사람은 조금 더 이해가 쉬울것이다. 단 한가지 형태의 필터만 사용하는 것이 아니라 N 개의 필터를 사용이 가능하다. 한 지점의 이미지에 100개의 필터를 적용하면 하나의 점의 데이터값이 100개의 각각의 필터를 적용한 값의 배열로 변환된다. 

합성곱 계층을 생성하기 위해서는 keras.layers.Dense 계층과 마찬가지로 keras.layers.Conv2D 로 생성되며 앞에 필터의 숫자와 패치의 크기를 입력해 주어야 한다. 

    model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape=(28,28,1)))

위의 코드는 28x28 의 이미지를 3x3 의 필터를 32가지를 적용하여 그 출력을 뽑아내라는 의미이다. 

### max pooling 계층

맥스풀링은 데이터의 차원을 감소시키는 효과를 지닌다. 풀링 영역이 2x2 이면 4개의 픽셀값중에 가장 큰 값을 대표값으로 선정해서 추출한다. 이는 합성곱에 의해 대폭 늘어난 데이터량을 감소시킴과 동시에 패턴이 특정 영역에 국한되지 안고 어느정도 유동적인 위치변화를 학습가능하게 해준다. 

    model.add(layers.MaxPooling2D((2,2))) 
    
위는 특정 2x2 의 영역에 대해 풀링을 수행하는 계층을 생성한다는 의미이다. 


### fllatten 계층

 cnn을 사용함에 있어서도 그 근간은 역전파 모델의 루틴을 사용한다. 그러나 합성곱 신경망의 계층은 3차원을 기반으로 구성된다. 그러므로 일반 역전파 모델과 결합하기 위해 2차원이나 3차원데이터를 1차원으로 변환하는 계층이 Flatten 이다. 

    model.add(layers.Flatten()) 
 


### 간단한 cnn 의 구성 예

다음은 28x28 의 이미지를 받아들여 처리하는 1계층의 합성곱과 맥스풀링으로 구성된 신경망의 사례이다. 

In [0]:
from keras import layers 
from keras import models
import keras

keras.backend.clear_session()
model = models.Sequential() 
model.add(layers.Conv2D(32,(5,5), activation = 'relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((3,3))) 

model.add(layers.Flatten())
model.add(layers.Dense(64,activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))


model.compile(optimizer = 'rmsprop',
                loss = 'categorical_crossentropy',
                metrics = ['acc'] )





<hr style="height:2px" > 

## Mnist 데이터를 CNN으로 처리해보기 

역전파 모델의 사례로 등장한 필기체 숫자인식 데이터 Mnist를 기억하는가?  CNN 은 사실 방대한 이미지를 오랜시간 학습함으로서 그 성능이 나오는 신경망이기에 수업시간중에 이를 완전히 실습해보기엔 쉽지안다. 하지만 기초적인 Mnist 같은 문제에도 사용이 가능하며 앞서 다뤘던 단순신경망 역전파 모델에 비해서는 더 나은 성능을 보여준다. 

### Mnist 데이터 로드 

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical 

(train_images, train_labels), (test_images, test_labels ) = mnist.load_data() 



### Mnist 데이터 변환 

In [0]:
train_x = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_x = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_y = to_categorical(train_labels)
test_y = to_categorical(test_labels)

### Mnist 데이터를 위의 예제에 적용하여 학습해보기 

In [0]:
import matplotlib.pyplot as plt
history = model.fit(train_x, train_y, epochs = 10, batch_size = 512,  validation_data = (test_x, test_y) ).history

### <font color = 'red'> 연습문제 4-1 : 학습 그래프 그려보기
    
위의 학습 과정을 이전시간에 했던것처럼  validation_data 와 함계 그 정확도의 변화를 도식화시켜보세요 


In [0]:
## 연습 4-1 의 코드를 여기에 작성해보세요 
plt.plot(history['loss'], label = 'loss')            
plt.plot(history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

plt.plot(history['acc'], label = 'acc')            
plt.plot(history['val_acc'], label = 'val_acc')
plt.legend()
plt.show()

print("last acc / val_acc : ", history['acc'][-1], "/",  history['val_acc'][-1])

### 실제 이미지를 적용하여 비교 
    
아래의 그림은 이전 수업에서 사용한 Mnist 역전파 모델에 실제로 직접 생성한 이미지를 적용하기 위해서 일련의 숫자이미지를 잘라서 입력하는데 사용한 코드를 살짝 수정한 것이다.  아래의 코드를 이용해 직접 숫자를 입력해서 그 결과를 볼 수 있다 

In [0]:
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt 

im = Image.open("t_nums2.png").convert('L')
im = np.array(im.getdata()).reshape(im.size[1], im.size[0])
plt.imshow(im, cmap = 'gray')
plt.show()
vsum = np.sum(im, axis = 0)

thresh = 0

s = -1 
w,h = 28,28

inputs = np.empty((0,28,28))


for i in range( im.shape[1] ):
    if s >= 0 and vsum[i] <= thresh :         
        narr = im[:,s:i].copy()      
        
        pad = np.zeros( (narr.shape[0],3) ) 
        #pady = np.zeros( (narr.3,shape[1]) ) 
                
        narr = np.hstack( (pad, narr, pad) ) 
        
        num = Image.fromarray(narr)
        num = np.array( num.resize((w,h)).getdata() )        
        
        inputs = np.vstack((inputs, num.reshape(1,w,h)))
        print(inputs.shape)
        
        s = -1
    elif s < 0 and vsum[i] > thresh : 
        s = i

if s > 0 :
    areas += im[a:,s:i].copy()

print( inputs[:,:,:,np.newaxis].shape )
    
print(inputs.shape)

output = model.predict(inputs[:,:,:,np.newaxis])
result = np.argmax(output, axis = 1)
print(result)


### <font color = 'red'> 연습문제 4-2 : 직접 만든 이미지로 Mnist 와 비교 네트워크 변경 후 
    
제공된 이미지로 Mnist 와 학습된 결과를 비교해보고 직접 손글씨 데이터를 만들어 성능의 비교를 해보세요 

<hr style="height:2px" > 

## 참고: Google Colab 

합성 신경망의 경우 사실 gpu의 도움이 없이는 간단한 예제도 이를 학습하는데 제법 시간이 걸리기 시작했을 것이다. 특히 네트워크를 여러가지로 변환해볼 경우에 10 epoch 만으로도 2-3분 이상이 걸려서는 사실 제대로된 연구가 어렵다. gpu의 도움을 저렴하게 얻는 방법중에 하나는 Google Colab 을 사용하는 것이다. google 은 TPU와 GPU를 지원하며 주피터 노트북상에서 여러 프로그램을 돌려볼 수 있도록 하는 Colab 이라는 무료 가상 플랫폼 서비스를 제공중이다.

https://colab.research.google.com

위의 링크에서 이를 이용할 수 있으니 참고하자.  참고로 파일을 업로드 하는 건 다음 코드를 코랩상에서 실행한 다음에 파일을 선택해서 업로드 해주면 된다. 



In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


<hr style="height:2px" > 

## Keggle Cats vs Dogs 학습

이번에는 보다 실용적인 데이터로 Keggle 에서 제공하는 데이터를 이용한 학습을 알아보겠다. keggle 은 이미지 인식 경연용 데이터를 제공하는데 대표적으로 개와 고양이의 이미지를 모아서 비교하는 데이터가 있다. 

    https://www.kaggle.com/c/dogs-vs-cats/data 

원본은 다음에 있지만 가입뿐만 아니라 전화번호까지 요구하므로 업로드된 어제자 따끈따근한 데이터를 쓰기로 하자.  

<a href = 'http://221.146.61.73:8888/edit/dnc/dogs-vs-cats.zip' >  download  </a>


다운로드 받은 이미지 파일을 풀면 train, test1 두가지 압축 파일이 보일 것이다. test1 은 테스트용 이미지인데 레이블링이 되어있지 안아 쓰기 불편하니 일단 train만 사용해 학습과 검증을 하도록 하겠다. 


### 데이터 제너레이터 

train 파일을 풀어보면 모든 이미지가 dog.1.jpg, dog.2.jpg ..  , cat.1.jpg, cat.2.jpg, .. 이런식으로 다루기 편하게 네이밍 되어있는 것을 볼 수 있다. 이런 파일들을 다룰때는 직접 읽어들여서 이것을 넘피 배열로 바꿔주는 것보다 keras 의 ImageDataGenerator 를 사용하면 편하다. 데이터 제너레이터는 디렉토리상의 파일들을 읽어서 자동으로 numpy 형식의 입력데이터로 변환해 줄 뿐만 아니라 데이터의 변조생성까지 지원하므로 이미지 학습시에는 필수에 가깝다. 

ImageDataGenerator 는 이미지 데이터 처리시 이미지를 디렉토리별로 클래스를 구분하는데 위의 예제의 경우 이미지를 train/dog, train/cat 으로 구분해주어야 한다. validation 까지 생각해서 디렉토리를 다음과 같이 구성해보자. 전체 학습은 시간이 오래걸리므로 1/10 정도의 데이터로 구성하는 것을 추천한다. 일단 본인인은 train : 2500 , test: 250 로 구성했 진행하도록 하겠다. 

    train_s
        dog
        cat 
    test_s
        dog
        cat


이제 데이터 제네레이터를 정의해보도록 하겠다. 이미지 사이즈는 속도까지 고려해 150x150 으로 정규화한다. 

In [0]:
from keras.preprocessing.image import ImageDataGenerator 

trdg = ImageDataGenerator(rescale = 1./255 )
ttdg = ImageDataGenerator(rescale = 1./255 )

train_gen = trdg.flow_from_directory( 
    "dnc/train_s",
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary') 

val_gen = ttdg.flow_from_directory(
    "dnc/test_s",
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary') 



이들을 수동으로 데이터를 얻어낼 경우 다음과 같이 for 문을 사용할 수 있다. 

In [0]:
for data_batch, labels_batch in train_gen:
    print(data_batch.shape, labels_batch.shape)
    break

모든 이미지가 batch size 에 맞춰서 20개의 150x150x3 배열과 그에 해당하는 레이블로 출력되는 것을 알 수 있을 것이다. 

### 모델 구성 
    
나머지는 지난시간의 mnist 와 거의 유사하지만 150x150 이기 때문에 더 많은 계층을 거쳐서 추상화 할 수 있다. input shape 가 3색을 사용하기 때문에 150,150,3 이라는 것에 유념해서 보도록 하자.  

아래는 교재에 실려있는 구조이고 실험결과에서도 무난한 성능을 나타낸 모델이다. 하지만 각각의 변형을 해보는 것도 좋다. 단 출력계층 이전의 Dense 계층의 숫자는 여유있게 주는 것을 추천한다. 

In [0]:
from keras import layers 
from keras import models
import keras

keras.backend.clear_session()
model = models.Sequential() 
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2))) 

model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))


model.add(layers.Flatten())
model.add(layers.Dense(512,activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = 'rmsprop',
                loss = 'binary_crossentropy',
                metrics = ['acc'] )


## 학습 
    
다음은 데이터 제네레이터를 사용해 학습을 하는 방법이다. 이를 돌려보고 직접 시각화하며 그 결과를 살펴보자. fit을 사용하는 일반적인 학습과 달리 generator 를 통합 학습은 fit_generator 라는 메소드를 사용한다. 기본적으로는 input_data, valdation_data 에 데이터 대신 제네레이터를 입력하는 점이 다르다. 그리고 generator 에 이미 batch_size 가 지정되어 있기 때문에 이를 사용하지 안고 대신 steps_per_epoch 라는 파라미터가 사용되는데 이는 한 epoch 를 30회의 배치로 나누어 처리한다는 의미로 이해하면 된다. 


In [0]:
history = model.fit_generator(train_gen, steps_per_epoch = 100,  epochs = 30, validation_data = val_gen, validation_steps = 50).history

이제 학습 결과를 도식화해보자. 

In [0]:
import matplotlib.pyplot as plt
plt.plot(history['loss'], label = 'loss')            
plt.plot(history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

plt.plot(history['acc'], label = 'acc')            
plt.plot(history['val_acc'], label = 'val_acc')
plt.legend()
plt.show()

print("last acc / val_acc : ", history['acc'][-1], "/",  history['val_acc'][-1])

아마 실제 적중률은 80% 정도에서 그 이후에는 과적합 상태로 흐르는 것을 볼 수 있을것이다.  더 세세한 설정과 전체 데이터를 기준으로 처리하면 더 높은 적중률을 얻을 수 있겠지만 일단은 이정도로 만족하자. 

### 실제 이미지 예측에 적용



In [0]:
from keras.preprocessing import image
from urllib.request import urlopen
import numpy as np 

URL = 'https://images.mypetlife.co.kr/content/uploads/2019/06/06202021/adorable-animal-baby-416088-1024x774.jpg'

#img = image.load_img('dnc/test1/16.jpg', target_size = (150,150))
img = image.load_img(urlopen(URL), target_size = (150,150))
x = image.img_to_array(img)[np.newaxis,:,:,:]
plt.imshow(img)

r = model.predict(x)
print(r)